# Fix problematic addresses that won't be geocoded 

## Setup

In [53]:

# Print all outputs from commnads in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#from urllib.request import Request, urlopen
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
import geocoder


In [68]:
# Import files 
df = pd.read_csv("/home/jae/analyzing-asian-american-latino-civic-infrastructure/processed_data/org_lat_logs.csv")

## Identify problematic addresses 

In [69]:
df.head()

,Address,F.year,latitude,longtitude,geo_method,group
0,"101 8th Street, Suite 101 Oakland, CA 94607",1974,NaN,NaN,osm,Asian
1,"1016 West Argyle Street Chicago, IL 60640",1981,41.970223,-87.837310,osm,Asian
2,"1031 25th St, San Diego",1974,32.716370,-117.140274,census,Asian
3,"1038 Post Street San Francisco, CA 94109",1970,37.786960,-122.418930,census,Asian
4,"1055 Wilshire Blvd., Suite 1475 Los Angeles, C...",1976,NaN,NaN,osm,Asian


In [70]:
# Find NAs

df['latitude'].isnull().head()

0     True
1    False
2    False
3    False
4     True
Name: latitude, dtype: bool

In [71]:
# The count of the problematic addresses in each group

df.loc[df['latitude'].isnull() & (df['group'] == "Asian")].shape[0] #19
df.loc[df['latitude'].isnull() & (df['group'] != "Asian")].shape[0] #12

19

12

In [72]:
# Index by NAs
pr_addrs = df['Address'].loc[df['latitude'].isnull()]

pr_addrs.head()

0           101 8th Street, Suite 101 Oakland, CA 94607
4     1055 Wilshire Blvd., Suite 1475 Los Angeles, C...
9        1250 Chambers Road Room 2403, Aurora, CO 80011
13    1511 Third Avenue Suite 914, Seattle, Washingt...
15    16161 Ventura Boulevard Suite 388, Encino, CA ...
Name: Address, dtype: object

In [82]:
# This index is useful to recode values later 
pr_index = np.array(pr_addrs.index)

pr_index

array([  0,   4,   9,  13,  15,  19,  23,  25,  27,  35,  52,  54,  55,
        56,  58,  62,  69,  70,  71,  72,  76,  81,  87,  97,  99, 128,
       130, 134, 141, 142, 143])

In [10]:
# Remove row index (this is important for iterations)

pr_addrs = pr_addrs.reset_index(drop = True)

pr_addrs.head()

0          101 8th Street, Suite 101 Oakland, CA 94607
1    1055 Wilshire Blvd., Suite 1475 Los Angeles, C...
2       1250 Chambers Road Room 2403, Aurora, CO 80011
3    1511 Third Avenue Suite 914, Seattle, Washingt...
4    16161 Ventura Boulevard Suite 388, Encino, CA ...
Name: Address, dtype: object

## Geocode using bing API

In [38]:

latlngs = []

for i in range(len(pr_addrs)):
    latlngs.append(geocoder.bing(pr_addrs[i], key = "bing_api_key"))

In [42]:
lat = []
long = []

for i in range(len(new_addrs)):
    lat.append(new_addrs[i].latlng[0])
    long.append(new_addrs[i].latlng[1])

In [51]:
# Check whether the for loop worked well 
None in lat
None in long 

False

False

In [86]:
# Update the data 

df['latitude'].loc[pr_index,] = np.array(lat)
df['latitude'].loc[pr_index,][0:5]

0     37.796717
4     34.021877
9     39.734480
13    47.609899
15    34.157100
Name: latitude, dtype: float64

In [87]:
df['longtitude'].loc[pr_index,] = np.array(long)
df['longtitude'].loc[pr_index,][0:5]

0    -122.266167
4    -118.499597
9    -104.808290
13   -122.338306
15   -118.485475
Name: longtitude, dtype: float64

In [88]:
df['geo_method'].loc[pr_index,] = "Bing"
df['geo_method'].loc[pr_index,][0:5]

0     Bing
4     Bing
9     Bing
13    Bing
15    Bing
Name: geo_method, dtype: object

## Export the result 

In [90]:
# Export the result as a csv  
df.to_csv("/home/jae/analyzing-asian-american-latino-civic-infrastructure/processed_data/org_lat_logs_fixed.csv")